###Обучить несколько разных моделей на наборе данных ССЗ (train_case2.csv): логрег, бустинг, лес и т.д - на ваш выбор 2-3 варианта
###при обучении моделей обязательно использовать кроссвалидацию,вывести сравнение полученных моделей по основным метрикам классификации: pr/rec/auc/f_score (можно в виде таблицы, где строки - модели, а столбцы - метрики)
###сделать выводы о том, какая модель справилась с задачей лучше других
###(опциональный вопрос) какая метрика (precision_recall_curve или roc_auc_curve) больше подходит в случае сильного дисбаланса классов? (когда объектов одного из классов намного больше чем другого).

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import precision_recall_curve, roc_curve, roc_auc_score, confusion_matrix
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin

In [2]:
df = pd.read_csv('train_case2.csv', sep=';')
df.head(3)

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1


In [3]:
# разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns='cardio'), 
                                                    df['cardio'], random_state=0)

In [4]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]


from sklearn.preprocessing import StandardScaler


continuos_cols = ['age', 'height', 'weight', 'ap_hi', 'ap_lo']
cat_cols = ['gender', 'cholesterol']
base_cols = ['gluc', 'smoke', 'alco', 'active']

continuos_transformers = []
cat_transformers = []
base_transformers = []

for cont_col in continuos_cols:
    transfomer =  Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())
            ])
    continuos_transformers.append((cont_col, transfomer))
    
for cat_col in cat_cols:
    cat_transformer = Pipeline([
                ('selector', ColumnSelector(key=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    cat_transformers.append((cat_col, cat_transformer))
    
for base_col in base_cols:
    base_transformer = Pipeline([
                ('selector', NumberSelector(key=base_col))
            ])
    base_transformers.append((base_col, base_transformer))

In [5]:
# объединим все наши трансформеры с помощью FeatureUnion

from sklearn.pipeline import FeatureUnion

feats = FeatureUnion(continuos_transformers+cat_transformers+base_transformers)
feature_processing = Pipeline([('feats', feats)])

feature_processing.fit_transform(X_train)

array([[-1.73391771,  0.6873301 ,  0.74843904, ...,  1.        ,
         0.        ,  1.        ],
       [-1.67343538,  0.07758923, -0.29640123, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.13738132,  1.17512278, -0.15708919, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 1.17775864,  1.17512278, -0.15708919, ...,  0.        ,
         0.        ,  1.        ],
       [-0.47190715, -1.38578883,  0.74843904, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.38174619,  0.56538192, -0.08743318, ...,  0.        ,
         0.        ,  1.        ]])

In [6]:
model_types = {'Логистичиская регрессия': LogisticRegression(random_state = 42),
               'Градиентный бустинг': GradientBoostingClassifier(random_state = 42),
               'Случайный лес': RandomForestClassifier(random_state = 42),
              }

models = []
for key in model_types:
    
    classifier = Pipeline([
        ('features',feats),
        ('classifier', model_types[key]),
    ])
    
    model = {'title': key, 'classifier': classifier}
    models.append(model)  

In [7]:
metrics_df = pd.DataFrame(columns=['model', 'Threshold', 'F-Score', 'Precision', 'Recall', 'Roc auc'])

In [8]:
for model in models:
  classifier = model['classifier']

  #запустим кросс-валидацию
  cv_scores = cross_val_score(classifier, X_train, y_train, cv=16, scoring='roc_auc')
  cv_score = np.mean(cv_scores)
  cv_score_std = np.std(cv_scores)
  print('CV score is {}+-{}'.format(cv_score, cv_score_std))

  #обучим пайплайн на всем тренировочном датасете
  classifier.fit(X_train, y_train)
  preds = classifier.predict_proba(X_test)[:, 1]

  #рассчитаем метрики
  precision, recall, thresholds = precision_recall_curve(y_test, preds)
  fscore = (2 * precision * recall) / (precision + recall)

  ix = np.argmax(fscore)
  roc_auc = roc_auc_score(y_test, preds)
  print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}, Roc auc ={roc_auc}')  
  metrics_df = metrics_df.append({
    'model': model['title'],
    'Threshold': thresholds[ix],
    'F-Score': fscore[ix],
    'Precision': precision[ix],
    'Recall': recall[ix],
    'Roc auc': roc_auc
  }, ignore_index=True)



CV score is 0.7867401104915408+-0.00852135511666111
Best Threshold=0.3869370891147378, F-Score=0.730, Precision=0.647, Recall=0.838, Roc auc =0.7840347790421852


<ipython-input-8-afe61c508eac>:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


CV score is 0.8025125910838183+-0.00707472977074522
Best Threshold=0.39494659702686624, F-Score=0.740, Precision=0.698, Recall=0.788, Roc auc =0.8026153641179974


<ipython-input-8-afe61c508eac>:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


CV score is 0.7734501681056019+-0.007171140345435727
Best Threshold=0.35, F-Score=0.719, Precision=0.643, Recall=0.816, Roc auc =0.7710366181802983


<ipython-input-8-afe61c508eac>:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


In [9]:
metrics_df

,model,Threshold,F-Score,Precision,Recall,Roc auc
0,Логистичиская регрессия,0.386937,0.730323,0.647431,0.837558,0.784035
1,Градиентный бустинг,0.394947,0.740248,0.697848,0.788134,0.802615
2,Случайный лес,0.350000,0.718863,0.642669,0.815553,0.771037


###Вывод: То, какая модель лучше справилась с задачей, будет определять цель задачи. Точность получилась выше у градиентного бустинга, но полнота выше у логистической модели.





###метрики ROC следует использовать, когда имеется примерно одинаковое количество наблюдений для каждого класса.
###метрики Precision-Recall следует использовать при наличии дисбаланса класса от среднего до большого.